In [ ]:
df_plot_standard = results[ results['Scaler'] == "standard"]
df_plot_standard_knn = df_plot_standard[df_plot_standard["Model"] == "Knn"]
df_plot_standard_knn

df_plot_minmax = results[ results['Scaler'] == "minmax"]
df_plot_minmax_knn = df_plot_minmax[df_plot_minmax["Model"] == "Knn"]
df_plot_minmax_knn

df_plot_quantil = results[ results['Scaler'] == "quantil"]
df_plot_quantil_knn = df_plot_quantil[df_plot_quantil["Model"] == "Knn"]
df_plot_quantil_knn

In [ ]:
plt.grid()
plt.plot(df_plot_standard_knn["k"],df_plot_standard_knn.Score,label = "Standars Scaler")
plt.plot(df_plot_minmax_knn.k,df_plot_minmax_knn.Score,label = "MinMax Scaler")
plt.plot(df_plot_quantil_knn.k,df_plot_quantil_knn.Score,label = "Quantil Scaler")
plt.xlabel("different k")
plt.ylabel("Score")
plt.title("Knn")
plt.legend()
plt.savefig("Bilder/Knn_with_different_k_sclaer")

In [ ]:
plt.grid()
plt.plot(df_plot_standard_knn.k,df_plot_standard_knn.Fittime,label = "Standars Scaler")
plt.plot(df_plot_minmax_knn.k,df_plot_minmax_knn.Fittime,label = "MinMax Scaler")
plt.plot(df_plot_quantil_knn.k,df_plot_quantil_knn.Fittime,label = "Quantil Scaler")
plt.xlabel("different k")
plt.ylabel("time [s]")
plt.title("Knn")
plt.legend()
plt.savefig("Bilder/FitTime_knn_with_different_k_sclaer")

In [ ]:
#k, scl = FindBestK_Scaler(results, "MLP")
#print(k,scl)
#k, scl = FindBestK_Scaler(results, "RM")
#print(k,scl)
k, scl = FindBestK_Scaler(results, "Knn")
print(k,scl)
df_results = results

In [ ]:
score_list = []
level = 4
alphas = 0.1
maxiter = 100
hidden = [10,20,50,100,200,400]
solver = ["adam","lbfgs"]
solver = ["adam"]
activation =["identity", "tanh", "relu"]
activation =["tanh"]
model = ["MLP"]
for mod in model:
    for lev in range(1,level+1):
        for h in hidden:
            for sol in solver:
                for act in activation:
                    results_pram = amazon_classificators.MLP_Search(alphas,act,sol,h,maxiter,X_train, X_valid, Y_train, Y_valid, lev)
                    best_score_MLP = best_score_MLP.append({'Model': mod, "HiddenLayer": h, "Level": lev, "Solver": sol, "Activation": act, "Score": results_pram["score"], "F1": np.mean(results_pram["f1"]), "Recall": np.mean(results_pram["recall"]), "Precision": np.mean(results_pram["precision"]), "Runtime": results_pram["time"]}, ignore_index=True)

In [ ]:
def FindBestK_Parameter(df):
    df_best = df[df["Score"] == max(df["Score"])]
    h = df_best["HiddenLayer"]
    lev = df_best["Level"]

    return h, lev

In [ ]:
h, lev = FindBestK_Parameter(best_score_MLP)
print(h.values[0], lev.values[0])

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(h.values[0],h.values[0],h.values[0]),alpha=0.1,activation="tanh",solver = "adam")
clf.fit(X_train, Y_train)#
#LC = clf.loss_curve_
Y_pred = clf.predict(X_valid)
score = accuracy_score(Y_valid, Y_pred)
print(score)

In [ ]:
from sklearn.metrics import f1_score
def Knn_Search(max_k,X_train, X_valid, Y_train, Y_valid):
    score_list = []
    erg = []
    for knn in range(1, max_k):
        model = KNeighborsClassifier(n_neighbors=knn).fit(X_train, Y_train)
        Y_pred = model.predict(X_valid)
        score = accuracy_score(Y_valid, Y_pred)
        score_list.append(score)
    knn = score_list.index(max(score_list))+1
    score = max(score_list)
    f1, recall, precision = Scores(Y_valid, Y_pred)
    erg = {
        "k_max": knn,
        "score": score,
        "f1": f1,
        "recall": recall,
        "precision": precision}
    print("")
    return erg, score_list

def Scores(Y_valid, Y_pred):
    f1 = f1_score(Y_valid, Y_pred, average='macro')
    cm = confusion_matrix(Y_valid, Y_pred)
    recall = np.diag(cm) / np.sum(cm, axis = 1)
    precision = np.diag(cm) / np.sum(cm, axis = 0)

    return f1, np.mean(recall), np.mean(precision)

In [ ]:
k_max = 100
model = "Knn"

results_pram, score_list = amazon_classificators.Knn_Search(k_max,X_train, X_valid, Y_train, Y_valid)
best_score_Knn = best_score_Knn.append({'Model': model, "k": results_pram["k_max"], "Score": results_pram["score"], "F1": np.mean(results_pram["f1"]), "Recall": np.mean(results_pram["recall"]), "Precision": np.mean(results_pram["precision"]), "Runtime": results_pram["time"]}, ignore_index=True)
    #best_params = amazon_classificators.FindBestScore(results)

In [ ]:
j = np.linspace(1,k_max-1,k_max-1)
plt.grid()
plt.plot(j,score_list)
print(np.max(score_list),score_list.index(np.max(score_list)))

In [ ]:
score_list = []

model = ["RM"]
estimators = [1, 10, 20, 100, 200, 500, 1000,1500,2000]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]


for es in estimators:
    for cri in criterion:
        for maxfe in max_features:
            results_pram = amazon_classificators.RM_Search(es,cri,maxfe,X_train, X_valid, Y_train, Y_valid)
            best_score_RM = best_score_RM.append({'Model': mod, "estimators": es, "criterion": cri, "max_features": maxfe, "Score": results_pram["score"], "F1": np.mean(results_pram["f1"]), "Recall": np.mean(results_pram["recall"]), "Precision": np.mean(results_pram["precision"]), "Runtime": results_pram["time"]}, ignore_index=True)

In [ ]:
#best_score_MLP = pd.DataFrame(columns=["Model", "HiddenLayer", "Level", "Solver", "Activation", "Score", "F1", "Recall", "Precision", "Runtime"])
#best_score_Knn = pd.DataFrame(columns=["Model", "k", "Score", "F1", "Recall", "Precision", "Runtime"])
#best_score_RM = pd.DataFrame(columns=["Model", "estimators", "criterion","max_features", "Score", "F1", "Recall", "Precision", "Runtime"])


In [ ]:
from sklearn.metrics import classification_report
target_names = pd.unique(target.Class)
print(classification_report(Y_valid, Y_pred, target_names=target_names))

cm = confusion_matrix(Y_valid, Y_pred)
fig = plt.figure(figsize=(70,70))
sns.heatmap(cm)

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(Y_valid, Y_pred, average='macro')
cm = confusion_matrix(Y_valid, Y_pred)
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print("f1: ", f1, "recall: ", np.mean(recall), "precision: ", np.mean(precision))

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Reds) :
    """classes are the possible classes, so e.g ["B","M"], s.t. the ordering matches the encoding"""
    plt.rcParams.update({'font.size': 12})
    num_samples = 1
    if normalize:
        num_samples = np.sum(cm)
    print("#",num_samples)
    plt.imshow(cm, interpolation = "nearest", cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 0)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2
    # itertools.product() gives all combinations of the iterables
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        string = cm[i, j]
        if normalize:
            string /= num_samples
            string = f"{string:.2f}"
        plt.text(j, i, string, horizontalalignment = "center", color="black", backgroundcolor="white")#= "white" if cm[i, j] > thresh else "black", )
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
plot_confusion_matrix(cm,pd.unique(target.Class))

In [ ]:
def plotLC(LC,best_params):
    plt.figure()
    plt.plot(LC)#,label="activation {} Hidden Layers {} solver {} ".format(best_params["alpha"],i,l))
    plt.grid()
    plt.legend()
    plt.xlabel("Iterations")best_score_Knn = pd.DataFrame(columns=["Model", "k", "Score", "F1", "Recall", "Precision"])
k, scl = FindBestK_Scaler(results, "Knn")

df_prep = amazon_processor.Preprocessing_Amazon(feature, target,feature_method = "kBest",scale_method = scl.values[0], k = k.values[0])
X_train, X_valid, Y_train, Y_valid = train_test_split(df_prep, target, test_size=.4, random_state = 42)
    plt.xlabel("Loss")
    plt.title("activation {} Hidden Layers {} solver {} ".format(best_params["mode"],best_params["h"],best_params["solver"]))
    #plt.savefig("figures//wine//LC//activation {} Hidden Layers {} solver {} ".format(best_params["mode"],best_params["h"],best_params["solver"]))
    plt.savefig("Bilder/activation {} Hidden Layers {} solver {} ".format(best_params["mode"],best_params["h"],best_params["solver"]))

In [ ]:
import itertools

# Confusion matrix 
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Reds) :
    """classes are the possible classes, so e.g ["B","M"], s.t. the ordering matches the encoding"""
    plt.rcParams.update({'font.size': 12})
    num_samples = 1
    if normalize:
        num_samples = np.sum(cm)
    print("#",num_samples)
    plt.imshow(cm, interpolation = "nearest", cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 0)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2
    # itertools.product() gives all combinations of the iterables
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        string = cm[i, j]
        if normalize:
            string /= num_samples
            string = f"{string:.2f}"
        plt.text(j, i, string, horizontalalignment = "center", color="black", backgroundcolor="white")#= "white" if cm[i, j] > thresh else "black", )
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    
# Show metrics 
def show_metrics(cm):
    """also returns the metrics as dict"""
    tn = cm[0,0]
    tp = cm[1,1]

    fp = cm[0,1]
    fn = cm[1,0]

    acc = (tp+tn) / (tp+tn+fp+fn)
    prec = tp / (tp+fp)
    rec = tp / (tp+fn)
    f1 = 2 * (prec * rec) / (prec + rec)
    print(f"Accuracy  =     {acc:.3f}")
    print(f"Precision =     {prec:.3f}")
    print(f"Recall    =     {rec:.3f}")
    print(f"F1_score  =     {f1:.3f}")
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

In [ ]:
from sklearn.metrics import mean_squared_error
def Statistic(Y_valid,Y_pred,name):
    Y_pred = pd.DataFrame(Y_pred, columns=['Class'])
    Class_description = []
    Class_description = pd.unique(Y_valid.Class)

    for i in range(len(Class_description)):
        Y_valid.Class[Y_valid.Class == Class_description[i]] = i+1
    
    Class_description = []
    Class_description = pd.unique(Y_pred.Class)

    for i in range(len(Class_description)):
        Y_pred.Class[Y_pred.Class == Class_description[i]] = i+1

    plt.figure()
    sns.distplot(Y_valid)
    sns.distplot(Y_pred, color="red")
    #plt.savefig("figures//wine//statistics//difference between prediction and validation {}".format(name))

    plt.figure()
    sns.distplot(Y_valid-Y_pred)
    #plt.savefig("figures//wine//statistics//total difference between prediction and validation {}".format(name))
    print("RMSE",sqrt(mean_squared_error(Y_valid, Y_pred)))

    Y_pred_Norm = Y_pred / np.linalg.norm(Y_pred)
    Y_valid_Norm = Y_valid / np.linalg.norm(Y_valid)

    print("NormedRMSE",sqrt(mean_squared_error(Y_pred_Norm, Y_valid_Norm)))

In [ ]:
Statistic(Y_valid, Y_pred,"Hallo")

#plotLC(LC,best_params)
cm_rf_N = confusion_matrix(Y_valid, Y_pred)
df_cm = pd.DataFrame(cm_rf_N, index = [i for i in target["Class"]],
                  columns = [i for i in target["Class"]])
plt.figure(figsize = (10,7))
#show_metrics(cm_rf_N)
#Statistic(Y_valid,Y_pred,"MLP GB_N")
plt.figure(figsize = (70,70))
sns.heatmap(cm_rf_N, annot=True)

from sklearn.metrics import plot_confusion_matrix
class_names =target.Class
disp = plot_confusion_matrix(clf, X_valid, X_valid,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues)

In [ ]:
from sklearn import svm
# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, Y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_valid, Y_valid,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    

    #fig = plt.figure(figsize=(70,70))
    #disp.ax_.set_title(title)
        
    #print(title)
    #print(disp.confusion_matrix)
#plt.show()

In [ ]:
plot_confusion_matrix(cm_rf_N,  ["B", "G"],
                          normalize = True,
                          title = 'Confusion matrix (knn)',
                          cmap = plt.cm.Blues)
metrics_rf_N = show_metrics(cm_rf_N)

In [ ]:
df_prep = amazon_processor.Preprocessing_Amazon(feature, target,feature_method = "PCA",scale_method = "standard")
df_prep

In [ ]:
#plt.rcParams.update({'font.size': 12})
#mpl.style.use('seaborn')
mpl.style.use('seaborn')
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
fig = plt.figure(figsize=(40,12))
#sns.histplot(count_classes,kde = False)
plt.grid()
plt.hist(cleaned["Class"],80,facecolor='g')
def PlotHist(df, xaxis,xlabel,ylabel,title,savename,bins):
    mpl.style.use('seaborn')
    plt.rc('xtick', labelsize=15) 
    plt.rc('ytick', labelsize=15) 
    fig = plt.figure(figsize=(40,12))
    #sns.histplot(count_classes,kde = False)
    plt.hist(cleaned["Class"],80,facecolor='g')
    plt.grid()
    #plt.savefig("Bilder/Class_distribution")
plt.savefig("Bilder/Class_distribution")

In [ ]:
#df_train['V1']=df_train['V1'].astype('category').cat.codes
#df_train['Class']=df_train['Class'].astype('category').cat.codes
#df_train.corr()

In [ ]:
from sklearn.metrics import f1_score
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
f1 = f1_score(y_true, y_pred, average='macro')

from sklearn.metrics import confusion_matrix
import numpy as np

from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average='macro')
cm = confusion_matrix(y_true, y_pred)
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print("f1: ", f1, "recall: ", np.mean(recall), "precision: ", np.mean(precision))

In [ ]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))